Random Bart Torvik Data Dump

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
folder = '/home/daniel/dev/marchMadness/data/12192024_BartTorvik/'

game_results_df = pd.read_csv(folder+'120924_Torvik_SeasonResults_clean.csv')
team_metrics_df = pd.read_csv(folder+'120924_Torvik_Trank_Data_clean.csv')

display(game_results_df)
display(game_results_df.columns.values)

display(team_metrics_df)
display(team_metrics_df.columns.values)

,Team,Opp.,W/L
0,Auburn,Duke,L
1,Villanova,Temple,W
2,Michigan St.,Minnesota,W
3,Oklahoma St.,Seton Hall,W
4,Duke,Auburn,W
...,...,...,...
3675,Evansville,Ohio St.,L
3676,Youngstown St.,Presbyterian,L
3677,Chicago St.,San Francisco,L
3678,Canisius,Maryland,L


array(['Team', 'Opp.', 'W/L'], dtype=object)

,Team,EFG%,EFGD%,FTR,FTRD,TOR,TORD,ORB,DRB,A?,...,3P%D,Blk,BlkD,Ast,AstD,3PR,3PRD,B?,C?,D?
0,UC Santa Barbara,56.3,46.3,27.0,26.6,18.9,16.6,29.7,25.7,67.4,...,29.7,7.9,7.7,52.8,44.7,45.0,39.6,66.4,75.7,73.1
1,Arkansas Pine Bluff,50.9,61.2,25.9,37.1,23.0,14.3,23.2,41.9,73.9,...,36.9,5.2,8.1,46.6,65.4,41.3,48.0,74.5,66.5,70.6
2,Toledo,49.9,55.2,32.7,25.4,13.5,16.1,24.7,31.9,70.9,...,34.6,7.5,9.2,47.4,44.1,30.7,38.3,72.0,71.4,69.8
3,Indiana,54.7,48.2,36.7,26.9,19.1,17.3,33.0,29.0,70.8,...,31.8,10.5,6.5,58.9,55.4,32.2,38.1,71.8,74.8,68.1
4,Louisiana,47.3,54.8,30.1,45.2,17.3,18.9,25.7,36.6,67.2,...,36.8,8.2,10.5,50,47.8,40.1,35.2,68.9,69.9,72.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,Fordham,49.3,51.7,32.2,34.5,19.0,18.8,30.9,27.8,72.3,...,36.0,7.3,8.6,51.7,55.5,40.7,30.4,71.9,70.4,69.5
360,Alabama St.,48.0,53.9,31.1,42.0,12.7,16.4,23.7,35.2,71.5,...,35.7,7.7,9.0,45.4,55.3,45.7,43.6,71.2,71.9,69.8
361,Wichita St.,49.7,49.6,39.3,25.4,14.0,16.0,25.4,29.6,71.5,...,32.6,12.0,10.0,47.5,58.2,32.0,42.7,72.4,72.3,59.7
362,Texas Tech,59.4,46.9,28.5,30.7,16.4,20.9,36.6,27.5,68.4,...,34.6,8.9,6.8,59.4,47.3,40.9,38.1,67.4,78.4,65.7


array(['Team', 'EFG%', 'EFGD%', 'FTR', 'FTRD', 'TOR', 'TORD', 'ORB',
       'DRB', 'A?', '2P%', '2P%D', '3P%', '3P%D', 'Blk', 'BlkD', 'Ast',
       'AstD', '3PR', '3PRD', 'B?', 'C?', 'D?'], dtype=object)

In [4]:
# Normalize team names to handle spaces and case issues
game_results_df['Team'] = game_results_df['Team'].str.strip().str.lower()
game_results_df['Opp.'] = game_results_df['Opp.'].str.strip().str.lower()
team_metrics_df['Team'] = team_metrics_df['Team'].str.strip().str.lower()

#clean weird ast error
team_metrics_df['Ast'] = pd.to_numeric(team_metrics_df['Ast'], errors='coerce')
team_metrics_df['Ast'] = team_metrics_df['Ast'].fillna(0)

# Initialize an empty list to store the combined results
combined_results = []

# Iterate through each game result
for index, row in game_results_df.iterrows():
    team_name = row['Team']
    opponent_name = row['Opp.']
    result = row['W/L']
    
    # Get team metrics for the current team
    team_row = team_metrics_df[team_metrics_df['Team'] == team_name]
    if not team_row.empty:
        team_metrics = team_row.iloc[0, 1:].values  # Get metrics for the team (skip the 'Team' column)
    else:
        print(f"Team {team_name} not found in team_metrics.csv")
        team_metrics = [0] * (len(team_metrics_df.columns) - 1)  # Default to zero if team not found
    
    # Get team metrics for the opponent
    opponent_row = team_metrics_df[team_metrics_df['Team'] == opponent_name]
    if not opponent_row.empty:
        opponent_metrics = opponent_row.iloc[0, 1:].values  # Get metrics for the opponent
    else:
        print(f"Opponent {opponent_name} not found in team_metrics.csv")
        opponent_metrics = [0] * (len(team_metrics_df.columns) - 1)  # Default to zero if opponent not found
    
    # Calculate the delta between team and opponent metrics
    delta_metrics = team_metrics - opponent_metrics
    
    # Combine the delta metrics with the win/loss result and append to the list
    result_row = list(delta_metrics) + [result]  # Add the 'W/L' result
    combined_results.append(result_row)

# Convert the combined results to a DataFrame
columns = [f"Metric_{i+1}" for i in range(len(delta_metrics))] + ['W/L']
combined_df = pd.DataFrame(combined_results, columns=columns)

combined_df.iloc[:, :-1] = combined_df.iloc[:, :-1].round(2)

# Save the result to a new CSV file
combined_df.to_csv('/home/daniel/dev/marchMadness/data/12192024_BartTorvik/combined_results.csv', index=False)

print("CSV file with team deltas and results saved as 'combined_results.csv'")


CSV file with team deltas and results saved as 'combined_results.csv'


Now that we have prepared the data: 


In [5]:
# Extract features (all columns except 'W/L')
X = combined_df.drop(columns=['W/L'])

# Extract target variable (W/L)
y = combined_df['W/L']

# Convert 'W/L' to numeric (W -> 1, L -> 0)
y = y.map({'W': 1, 'L': 0})

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Scale the features
X_scaled = scaler.fit_transform(X)



In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

LogisticRegression()

In [7]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print classification report for precision, recall, F1 score, etc.
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.76
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.76      0.76       363
           1       0.76      0.76      0.76       373

    accuracy                           0.76       736
   macro avg       0.76      0.76      0.76       736
weighted avg       0.76      0.76      0.76       736

Confusion Matrix:
[[276  87]
 [ 90 283]]


Create list of teams

In [8]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest model
rf_model = RandomForestClassifier()

# Train the model
rf_model.fit(X_train, y_train)

# Evaluate the Random Forest model
y_pred_rf = rf_model.predict(X_test)
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.2f}")

Random Forest Accuracy: 0.75


In [9]:
from sklearn.svm import SVC

# Initialize the SVM model
svm_model = SVC()

# Train the model
svm_model.fit(X_train, y_train)

# Evaluate the SVM model
y_pred_svm = svm_model.predict(X_test)
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")

SVM Accuracy: 0.75


In [10]:
from sklearn.neural_network import MLPClassifier

# Initialize a simple neural network model
nn_model = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=1000)

# Train the model
nn_model.fit(X_train, y_train)

# Evaluate the neural network model
y_pred_nn = nn_model.predict(X_test)
print(f"Neural Network Accuracy: {accuracy_score(y_test, y_pred_nn):.2f}")

Neural Network Accuracy: 0.73


In [11]:
teams = game_results_df['Team']

import itertools

# Generate all possible pairings (matchups) between teams
matchups = list(itertools.combinations(teams, 2))

In [12]:
def calculate_team_delta(team1, team2, team_metrics_df):
    if team1 not in team_metrics_df['Team'].values:
        print(f"Warning: {team1} not found in team metrics data.")
        return None
    if team2 not in team_metrics_df['Team'].values:
        print(f"Warning: {team2} not found in team metrics data.")
        return None

    # Get the metrics for both teams
    team1_metrics = team_metrics_df[team_metrics_df['Team'] == team1].drop(columns='Team')
    team2_metrics = team_metrics_df[team_metrics_df['Team'] == team2].drop(columns='Team')
    
    # Calculate the delta (difference in metrics)
    delta = team1_metrics.values - team2_metrics.values
    
    return delta.flatten()

In [14]:
#add in progress counter
from tqdm import tqdm



# Initialize a list to store the predictions
prediction_matrix = []


import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")


counter = 0

# Loop over all matchups
for team1, team2 in tdqm(matchups, desc="Progress left":
    counter = counter + 1
    if(counter % 1000 == 0):
        print(counter + "/6769360")
    
    # Calculate the delta for the current matchup
    delta = calculate_team_delta(team1, team2, team_metrics_df)
    
    # Skip matchups if delta calculation fails
    if delta is None:
        continue
    
    # Scale the delta if necessary (since we scaled the training data earlier)
    delta_scaled = scaler.transform([delta])
    
    # Use the trained model to predict the probability of a win
    win_probability = model.predict_proba(delta_scaled)[0][1]  # [0][1] is the probability of class '1' (win)
    
    # Append the prediction to the result list
    prediction_matrix.append([team1, team2, win_probability])

# Convert the predictions to a DataFrame
print(counter)
predictions_df = pd.DataFrame(prediction_matrix, columns=['Team1', 'Team2', 'Win Probability'])

# Round probabilities to a desired precision for readability (optional)
predictions_df['Win Probability'] = predictions_df['Win Probability'].round(3)


combined_df.to_csv('/home/daniel/dev/marchMadness/data/12192024_BartTorvik/predictions.csv', index=False)


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [113]:
print(len(matchups))


6769360
